In [2]:
import glob
from IPython.display import HTML
png_list = glob.glob("*.png")
image_list = ""
for png in png_list:
    image_list+=f"<image src='{png}' width='5px' >"
display(HTML('<p>'+image_list+'</p>'))

In [3]:
from IPython.display import display, Image, clear_output, HTML
import time
import random
import pandas as pd

In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from jupyter_ui_poll import ui_events

In [5]:
import requests
from bs4 import BeautifulSoup
import json

def send_to_google_form(data_dict, form_url):
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [6]:
image_dict = {}
for u in range(64):
    u += 1
    image_dict[f"i{u}"] = f"./ANS_Pics/ANS{u}.jpg"

image_list = []
for u in range(64):
    u += 1
    image_list.append(f"i{u}")

shuffled_image_list = image_list.copy()
random.shuffle(shuffled_image_list)

data_type = {"76" : ['i2', 'i6', 'i9', 'i10', 'i17', 'i20', 'i25', 'i29', 'i35', 'i39', 'i44', 'i46', 'i51', 'i53', 'i57', 'i60'],
             "43" : ['i3', 'i5', 'i8', 'i12', 'i14', 'i16', 'i19', 'i21', 'i23', 'i26', 'i28', 'i30', 'i34', 'i36', 'i40', 'i41', 'i47', 'i48', 'i49', 'i55', 'i56', 'i58', 'i61', 'i63'],
             "98" : ['i7', 'i13', 'i22', 'i32', 'i37', 'i43', 'i50', 'i64'],
             "109": ['i1', 'i4', 'i11', 'i15', 'i18', 'i24', 'i27', 'i31', 'i33', 'i38', 'i42', 'i45', 'i52', 'i54', 'i59', 'i62']
            }
Right = []
Left = []

for i in range(32):
    i += 1
    Right.append(f"i{i}")

for i in range(32):
    i += 33
    Left.append(f"i{i}")

In [7]:
test_info = {
    'type': '',
    'choice': '',
    'time': '',
}

In [8]:
def wait_for_response (timeout = -1, interval = 0.001, max_rate = 20, allow_interrupt = True):
    start = time.time()
    test_info['time'] = -1
    test_info['choice']= ""
    test_info['type'] = ""
    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            ui_poll(n_proc)

            if (timeout != -1) and (time.time() > start + timeout):
                keep_looping = False
                
            if allow_interrupt==True and test_info['choice']!="":
                keep_looping = False

            time.sleep(interval)

    return test_info

In [9]:
def register_event(btn):
    test_info['type'] = 'click'
    test_info['choice'] = btn.description
    test_info['time'] = time.time()
    
    return test_info

In [10]:
def ask_for_consent():
    data_consent_info = """DATA CONSENT INFORMATION:

    Please read:

    we wish to record your response data

    to an anonymised public data repository.

    Your data will be used for educational teaching purposes

    practising data analysis and visualisation.

    Please type yes in the box below if you consent to the upload."""

    print(data_consent_info)
    result = input("> ")
    if result == "yes":

        print("Thanks for your participation.")

        print("Please contact philip.lewis@ucl.ac.uk")

        print("If you have any questions or concerns")

        print("regarding the stored results.")

    else:

        raise(Exception("User did not consent to continue test."))

    return

In [11]:
def run_test():
    
    start = widgets.Button (description = "Start")
    left = widgets.Button (description = "Left")
    right = widgets.Button (description = "Right")
    
    start.on_click(register_event)
    left.on_click(register_event)
    right.on_click(register_event)

    ask_for_consent()
    time.sleep(3)
    clear_output(wait = False)
    
    id_instructions = """

    Enter your anonymised ID
    
    To generate an anonymous 4-letter unique user identifier please enter:
    
    - two letters based on the initials (first and last name) of a childhood friend
    
    - two letters based on the initials (first and last name) of a favourite actor / actress
    
    e.g. if your friend was called Peter Pan and film star was Brad Pitt
    
    then your unique identifer would be PPBP"""
    
    print(id_instructions)
    user_id = input("> ")
    time.sleep(0.5)
    print("Please enter your age:")
    age = input("> ")
    time.sleep(0.5)
    print("Please enter your gender (M for Male, F for Female, O for Others):")
    gender = input(">")
    time.sleep(0.5)
    clear_output(wait = False)
    time.sleep(1)

    welcome_word = HTML("""<span style = "color: green;">This is an approximate number system (ANS) test. <br><br>
    For each trial, a test image will be shown for 0.75 seconds. <br><br>
    2 buttons will appear on the top, displaying "Left" and "Right". <br><br>
    You need to choose the side with more dots and click on the corresponding button in 3 seconds.<br><br></span>""")
    start_guide = HTML("""<span style = "color: red;">Click the start button when you are ready to go.</span>""")
    display(welcome_word, start_guide)
    
    panel = widgets.HBox([start])
    panel1 = widgets.HBox([left, right])
    display(panel)
    
    result = wait_for_response()
    clear_output(wait = False)
    time.sleep(0.5)
    start_time = time.time()
    
    Judge = {
            "C76":0,
            "C43":0,
            "C98":0,
            "C109":0,
            "W76":0,
            "W43":0,
            "W98":0,
            "W109":0
           }

    no_click = 0
    total_correct = 0
    total_wrong = 0
    total_response_time = 0
    n = 0
    
    for i in shuffled_image_list:
        display(Image(image_dict[i], width = 500))
        time.sleep(0.75)
        clear_output(wait = False)
        start = time.time()
        display(panel1)
        result = wait_for_response(timeout=3, allow_interrupt=True)
        
        choice = result['choice']
        if choice == "":
            print("User did not click in time")
            no_click += 1
        else:
            u = i
            if choice == "Left":
                if u in Left:
                    for i in data_type:
                        if u in data_type[i]:
                            Judge[f"C{i}"]+= 1
                            total_correct += 1
                else:
                    for i in data_type:
                        if u in data_type[i]:
                            Judge[f"W{i}"]+= 1
                            total_wrong += 1
            elif choice == "Right":
                if u in Right:
                    for i in data_type:
                        if u in data_type[i]:
                            Judge[f"C{i}"]+= 1
                            total_correct += 1
                else:
                    for i in data_type:
                        if u in data_type[i]:
                            Judge[f"W{i}"]+= 1
                            total_wrong += 1
            clear_output(wait = False)
            end = time.time()
            total_response_time += end - start
            n += 1
        time.sleep(1.5)
        clear_output(wait = False)

    end_time = time.time()
    time_taken = end_time - start_time
    average = total_response_time/n
    correct_rate = total_correct/64
    
    C76 = Judge["C76"]
    C43 = Judge["C43"]
    C98 = Judge["C98"]
    C109 = Judge["C109"]
    W76 = Judge["W76"]
    W43 = Judge["W43"]
    W98 = Judge["W98"]
    W109 = Judge["W109"]
    
    outcome = {
        "user_id": user_id,
        "gender": gender,
        "age": age,
        "76 Correct": C76,
        "43 Correct": C43,
        "98 Correct": C98,
        "109 Correct": C109,
        "76 Wrong": W76,
        "43 Wrong": W43,
        "98 Wrong": W98,
        "109 Wrong": W109,
        "Not clicked in time": no_click,
        "Total Correct": total_correct,
        "Total Wrong": total_wrong,
        "Correct Rate": correct_rate,
        "Time Taken": time_taken,
        "Average response time": average
            }
    
    form_url = "https://docs.google.com/forms/d/e/1FAIpQLSfybwZCx7dJd8O9T_gNb3Am11ghlJK77FL3OEGrX93vE9JQ2Q/viewform?usp=sf_link"
    send_to_google_form(outcome, form_url)

    time.sleep(0.5)
    print(f"""Thanks for taking the test :D.
    You took {round(time_taken/60,2)} minutes to complete the test.
    The average response time is {round(average, 2)} seconds.
    Your score is {total_correct} out of 64.""")

    return

In [12]:
run_test()

Thanks for taking the test :D.
    You took 2.9 minutes to complete the test.
    The average response time is 0.46 seconds.
    Your score is 57 out of 64.
